In [1]:
# removes annoying deprecation warnings 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import pandas as pd
from google.cloud import bigquery
from bq_helper import BigQueryHelper #third party library to translate google query data to dataframe
import matplotlib.pyplot as plt
import os, sys

sys.path.insert(0, './../utils/')

# custom files 
import random_forest_regressor as rfr
import utilities as util
import validation as cv

from sklearn.model_selection import train_test_split
from sklearn import cross_validation
%matplotlib inline

with open('../../key.txt') as f:
    content = f.readlines()

#add your own key here 
os.environ['GOOGLE_APPLICATION_CREDENTIALS']= content[0]

/home/kristina/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# use columns corresponding to relatively petty crimes
SF_CRIME_QUERY = """
        SELECT
          COUNT( DISTINCT unique_key) as count,
          CONCAT(cast(EXTRACT(YEAR FROM SFCrimeData.timestamp) AS string), 
          cast(EXTRACT(MONTH FROM SFCrimeData.timestamp) AS string), 
          cast(EXTRACT(DAY FROM SFCrimeData.timestamp) AS string)) AS date,
          descript as description
        FROM
          `bigquery-public-data.san_francisco_sfpd_incidents.sfpd_incidents` AS SFCrimeData
        WHERE category = "OTHER OFFENSES"
        GROUP BY descript, date
        ORDER BY date DESC
        """
bq_assistant_SF_crime = BigQueryHelper("bigquery-public-data", "san_francisco_sfpd_incidents.sfpd_incidents")
df_SF_crime = bq_assistant_SF_crime.query_to_pandas(SF_CRIME_QUERY)

In [3]:
set(df_SF_crime['description'])

{'ABORTION',
 'ACTS AGAINST PUBLIC TRANSIT',
 'ADVERTISING DISTRIBUTORS PERMIT VIOLATION',
 'AFFIXING ADVERTISMENTS TO POLES',
 'AGGRESSIVE SOLICITING',
 'AID OR HARBOR FELON',
 'ANIMAL, FIGHTING',
 'ANIMAL, WITHOUT PROPER CARE OR ATTENTION',
 'BATHROOM HOLE, LOOKING THROUGH',
 'BEYOND PARENTAL CONTROL',
 'CHOP SHOP, OWN OR OPERATE',
 'COMPUTER SYSTEM, ACCESSING, COPYING, OR DAMAGING',
 'CONSPIRACY',
 'CONTRIBUTING TO THE DELINQUENCY OF MINOR',
 'CRUELTY TO ANIMALS',
 'CURFEW VIOLATION',
 'DANGER OF LEADING IMMORAL LIFE',
 'DEFRAUDING TAXI DRIVER',
 'DESTITUTE MINOR',
 'DISCHARGING OFFENSIVE OR INJURIOUS SUBSTANCE IN PUBLIC AREA',
 'DISPLAY & SALE OF SPRAY PAINT & MARKER PENS',
 'DISRUPTS SCHOOL ACTIVITIES',
 'DOG OR CAT, ABANDONMENT OF',
 'DOG, BARKING',
 'DOG, FIGHTING;  OWNING, FIGHTING, OR ATTENDING FIGHT',
 'DOG, STRAY OR VICIOUS',
 'DRIVERS LICENSE, SUSPENDED OR REVOKED',
 'DRIVES VEHICLE ALONG TRACK OF RAILROAD',
 'DRIVING, DRAG RACING',
 'DRIVING, RECKLESS, WITH INJURY',
 'DRUG